In [2]:
from huggingface_hub import snapshot_download

local_dir = snapshot_download(
    repo_id="ertiaM/Anomaly_Detection_in_Surveillance_Videos",
    repo_type="dataset",
    local_dir="/content/drive/MyDrive/Anomaly_Detection_in_Surveillance_Videos"
)
print(f"Dataset downloaded to: {local_dir}")


Fetching 952 files:   0%|          | 0/952 [00:00<?, ?it/s]

Dataset downloaded to: /content/drive/MyDrive/Anomaly_Detection_in_Surveillance_Videos


In [6]:
!pip install opencv-python

In [7]:
import os

dataset_path = "/content/drive/MyDrive/Anomaly_Detection_in_Surveillance_Videos"

video_files = []
for root, _, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(".mp4"):  # Assuming videos are in mp4 format
            video_files.append(os.path.join(root, file))

print(f"Found {len(video_files)} video files.")

Found 950 video files.


In [8]:
import cv2
import numpy as np

def preprocess_video(video_path, output_dir, target_size=(224, 224), frame_sample_interval=1):
    """
    Preprocesses a single video by resizing frames and sampling.

    Args:
        video_path (str): Path to the input video file.
        output_dir (str): Directory to save the preprocessed frames.
        target_size (tuple): Desired size of the frames (width, height).
        frame_sample_interval (int): Take every nth frame.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return

    video_name = os.path.splitext(os.path.basename(video_path))[0]
    video_output_dir = os.path.join(output_dir, video_name)
    os.makedirs(video_output_dir, exist_ok=True)

    frame_count = 0
    saved_frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_sample_interval == 0:
            resized_frame = cv2.resize(frame, target_size)

            # Save frame
            frame_filename = os.path.join(video_output_dir, f"frame_{saved_frame_count:04d}.jpg")
            cv2.imwrite(frame_filename, resized_frame)
            saved_frame_count += 1

        frame_count += 1

    cap.release()
    print(f"Processed {video_path}: Saved {saved_frame_count} frames to {video_output_dir}")

In [9]:
preprocessed_output_dir = "/content/drive/MyDrive/Preprocessed_Surveillance_Videos"
os.makedirs(preprocessed_output_dir, exist_ok=True)

for video_file in video_files:
    preprocess_video(video_file, preprocessed_output_dir, target_size=(224, 224), frame_sample_interval=5)

Processed /content/drive/MyDrive/Anomaly_Detection_in_Surveillance_Videos/Anomaly-Videos-Part-1/Abuse/Abuse005_x264.mp4: Saved 190 frames to /content/drive/MyDrive/Preprocessed_Surveillance_Videos/Abuse005_x264
Processed /content/drive/MyDrive/Anomaly_Detection_in_Surveillance_Videos/Anomaly-Videos-Part-1/Abuse/Abuse007_x264.mp4: Saved 230 frames to /content/drive/MyDrive/Preprocessed_Surveillance_Videos/Abuse007_x264
Processed /content/drive/MyDrive/Anomaly_Detection_in_Surveillance_Videos/Anomaly-Videos-Part-1/Abuse/Abuse002_x264.mp4: Saved 173 frames to /content/drive/MyDrive/Preprocessed_Surveillance_Videos/Abuse002_x264
Processed /content/drive/MyDrive/Anomaly_Detection_in_Surveillance_Videos/Anomaly-Videos-Part-1/Abuse/Abuse001_x264.mp4: Saved 546 frames to /content/drive/MyDrive/Preprocessed_Surveillance_Videos/Abuse001_x264
Processed /content/drive/MyDrive/Anomaly_Detection_in_Surveillance_Videos/Anomaly-Videos-Part-1/Abuse/Abuse003_x264.mp4: Saved 740 frames to /content/drive/

In [10]:
import os

preprocessed_output_dir = "/content/drive/MyDrive/Preprocessed_Surveillance_Videos"

preprocessed_video_dirs = [d for d in os.listdir(preprocessed_output_dir) if os.path.isdir(os.path.join(preprocessed_output_dir, d))]

if preprocessed_video_dirs:
    print(f"Found {len(preprocessed_video_dirs)} preprocessed video directories.")
    print("Listing contents of the first 5 directories:")
    for i, video_dir in enumerate(preprocessed_video_dirs[:5]):
        video_dir_path = os.path.join(preprocessed_output_dir, video_dir)
        frames = [f for f in os.listdir(video_dir_path) if f.endswith(".jpg")]
        print(f"- {video_dir}: {len(frames)} frames")
else:
    print("No preprocessed video directories found.")

Found 950 preprocessed video directories.
Listing contents of the first 5 directories:
- Abuse005_x264: 190 frames
- Abuse007_x264: 230 frames
- Abuse002_x264: 173 frames
- Abuse001_x264: 546 frames
- Abuse003_x264: 740 frames
